### Notebook that explores how to read parquet files and upload to GCS bucket

The goal is to read from the page https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page the data for all 2022 and upload every month in a bucket

In [7]:
import io
import os
import requests
import pandas as pd
from google.cloud import storage

In [8]:
init_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/"
BUCKET = os.environ.get("GCP_GCS_BUCKET", "terraform-stavros-bucket")

In [9]:
def upload_to_gcs(bucket, object_name, local_file):
    """
    Ref: https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
    """
    # # WORKAROUND to prevent timeout for files > 6 MB on 800 kbps upload speed.
    # # (Ref: https://github.com/googleapis/python-storage/issues/74)
    # storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB
    # storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB

    client = storage.Client()
    bucket = client.bucket(bucket)
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)

In [10]:
def web_to_gcs(year, service):
    for i in range(12):
        
        # sets the month part of the file_name string
        month = '0'+str(i+1)
        month = month[-2:]

        # csv file_name
        file_name = f"{service}_tripdata_{year}-{month}.parquet"

        # download it using requests via a pandas df
        request_url = f"{init_url}{service}/{file_name}"
        r = requests.get(request_url)
        open(file_name, 'wb').write(r.content)
        print(f"Local: {file_name}")

        # upload it to gcs 
        upload_to_gcs(BUCKET, f"{service}/{file_name}", file_name)
        print(f"GCS: {service}/{file_name}")


In [11]:
web_to_gcs('2022', 'green')

Local: green_tripdata_2022-01.parquet
GCS: green/green_tripdata_2022-01.parquet
Local: green_tripdata_2022-02.parquet
GCS: green/green_tripdata_2022-02.parquet
Local: green_tripdata_2022-03.parquet
GCS: green/green_tripdata_2022-03.parquet
Local: green_tripdata_2022-04.parquet
GCS: green/green_tripdata_2022-04.parquet
Local: green_tripdata_2022-05.parquet
GCS: green/green_tripdata_2022-05.parquet
Local: green_tripdata_2022-06.parquet
GCS: green/green_tripdata_2022-06.parquet
Local: green_tripdata_2022-07.parquet
GCS: green/green_tripdata_2022-07.parquet
Local: green_tripdata_2022-08.parquet
GCS: green/green_tripdata_2022-08.parquet
Local: green_tripdata_2022-09.parquet
GCS: green/green_tripdata_2022-09.parquet
Local: green_tripdata_2022-10.parquet
GCS: green/green_tripdata_2022-10.parquet
Local: green_tripdata_2022-11.parquet
GCS: green/green_tripdata_2022-11.parquet
Local: green_tripdata_2022-12.parquet
GCS: green/green_tripdata_2022-12.parquet
